## Allocation of service departments to user departments ##

---

First load some modules and create some functions for formatting

In [1]:
import numpy as np
import pandas as pd


def total_allocations(df, lyst):
    df = df.copy()
    tot_row =  df.sum(numeric_only=True)
    tot_row.loc[lyst] = 0
    df.loc['Total'] = tot_row
    df['Total of rows'] = df.sum(axis=1)
    #print(df.to_markdown(floatfmt=",.3f"))
    df = df.fillna(0)
    df.to_clipboard()
    return df.loc[~(df==0).all(axis=1)]


def direct_allocations(df):
    df = df.copy()
    df.loc['Total'] = df.sum(numeric_only=True)
    df['Total'] = df.sum(axis=1)
    df = df.fillna(0)
    df = df.loc[~(df==0).all(axis=1)]
    print(df.to_markdown(floatfmt=",.3f"))
    df.to_clipboard()
    return df


def make_pretty(styler):
    styler.background_gradient(axis=None, vmin=-1, vmax=1, cmap="Blues_r").format(precision=2)
    return styler


def make_prettier(styler):
    styler.background_gradient(axis=None, vmin=-1000, vmax=7000, cmap="Blues_r").format(precision=2)
    styler.highlight_null('white')
    return styler


In [ ]:
# Example from Zimmerman
dfs = pd.read_csv("costs_zimm.csv").rename(columns={'Unnamed: 0' : "Departments"}).set_index("Departments").astype(float).dropna(how='all', axis=1).dropna(how='all')
list(dfs)
upstream = ['TC', 'IT']
downstream = ['Cars', 'Trucks']
dept_names = upstream + downstream
dfs.style.pipe(make_pretty)


In [ ]:
# Example from my Lecture
dfs = pd.read_csv("costs_lecture.csv").rename(columns ={'Unnamed: 0' : "Departments"}).set_index("Departments").astype(float)
list(dfs)

upstream = ['Maint', 'HR']
downstream = ['Assembly', 'Finishing']
dept_names = upstream + downstream
dfs.style.pipe(make_pretty)

In [ ]:
# Example from Zimmerman: fidelity
dfs = pd.read_csv("cost_fidelity.csv").rename(columns ={'Unnamed: 0' : "Departments"}).set_index("Departments").astype(float)
dfs
list(dfs)

upstream = ['Telco', 'Info', 'Build', 'TrainDevt', 'HR']
downstream = ['Retail', 'Commercial', 'Credit']
dept_names = upstream + downstream
dfs.style.pipe(make_pretty)

In [ ]:
# Example from Zimmerman: Murray
dfs = pd.read_csv("costs_murray.csv").rename(columns ={'Unnamed: 0' : "Departments"}).set_index("Departments").astype(float)
dfs
list(dfs)

upstream = ['A', 'B', 'C']
downstream = ['D1', 'D2', 'D3']
dept_names = upstream + downstream
dfs.style.pipe(make_pretty)


In [ ]:
# Example from Zimmerman: Dovovan Water first
dfs = pd.read_csv("costs_donovan_water.csv").rename(columns ={'Unnamed: 0' : "Departments"}).set_index("Departments").astype(float)
dfs
list(dfs)

upstream = ['Water','Electricity']
downstream = ['Ingots', 'Steel']
dept_names = upstream + downstream
dfs.style.pipe(make_pretty)

In [ ]:
# Example from Zimmerman: Dovovan Electricity first
dfs = pd.read_csv("costs_donovan_elec.csv").rename(columns ={'Unnamed: 0' : "Departments"}).set_index("Departments").astype(float)
dfs
list(dfs)

upstream = ['Electricity', 'Water']
downstream = ['Ingots', 'Steel']
dept_names = upstream + downstream
dfs.style.pipe(make_pretty)

In [2]:
# Example from Schoute (VU)
dfs = pd.read_csv("costs.csv").rename(columns ={'Unnamed: 0' : "Departments"}).set_index("Departments")
list(dfs)
upstream = ['HR', 'PD', 'IT', 'MA']
downstream = ['LT', 'VG', 'ER']
dept_names = upstream + downstream
dfs.style.pipe(make_pretty)


,HR,PD,IT,MA,LT,VG,ER
Departments,,,,,,,
Cost,800000.00,600000.00,400000.00,200000.00,2000000.00,1800000.00,1200000.00
HR,-1.00,0.15,0.25,0.10,0.40,0.05,0.05
PD,0.25,-1.00,0.20,0.15,0.35,0.00,0.05
IT,0.20,0.10,-1.00,0.05,0.00,0.45,0.20
MA,0.10,0.05,0.15,-1.00,0.05,0.35,0.30
LT,0.00,0.00,0.00,0.00,1.00,0.00,0.00
VG,0.00,0.00,0.00,0.00,0.00,1.00,0.00
ER,0.00,0.00,0.00,0.00,0.00,0.00,1.00


So, HR's cost are allocated to the departments downstream (actually to the right). 15 percent goes to PD, 25 percent to IT, ... and 5 percent to ER.

### Direct Method ###

---

First make a vector with the cost data from the upper row of the table above:

In [3]:
df_cost = dfs[dept_names].loc["Cost"].mul(0.001)
df_cost

HR     800.0
PD     600.0
IT     400.0
MA     200.0
LT    2000.0
VG    1800.0
ER    1200.0
Name: Cost, dtype: float64

Make a matrix with the weights from the lower rows of the table above, i.e., **only** for the downstream user departments. This is typical for the direct method.

In [4]:
df_wght_matrix = dfs[downstream].loc[dept_names]
df_wght_matrix.style.pipe(make_pretty)

,LT,VG,ER
Departments,,,
HR,0.40,0.05,0.05
PD,0.35,0.00,0.05
IT,0.00,0.45,0.20
MA,0.05,0.35,0.30
LT,1.00,0.00,0.00
VG,0.00,1.00,0.00
ER,0.00,0.00,1.00


Then re-weigh the weights so that they sum up to 100%:

In [5]:
df_wght_matrix = df_wght_matrix.assign(Total=df_wght_matrix.sum(axis=1))
df_wght_matrix = df_wght_matrix[downstream].div(df_wght_matrix["Total"], axis=0)
df_wght_matrix.dropna(how='all').style.pipe(make_pretty)

,LT,VG,ER
Departments,,,
HR,0.80,0.10,0.10
PD,0.88,0.00,0.13
IT,0.00,0.69,0.31
MA,0.07,0.50,0.43
LT,1.00,0.00,0.00
VG,0.00,1.00,0.00
ER,0.00,0.00,1.00


As you can see, all user departments are allocated 100 percent of the service departments:

In [6]:
df_wght_matrix.sum(axis=1)

Departments
HR    1.0
PD    1.0
IT    1.0
MA    1.0
LT    1.0
VG    1.0
ER    1.0
dtype: float64

---

Lastly: **allocate** by multipling the keys by the cost vector:

In [7]:
df_direct = df_wght_matrix.mul(df_cost, axis=0)
df_direct

,LT,VG,ER
Departments,,,
HR,640.000000,80.000000,80.000000
PD,525.000000,0.000000,75.000000
IT,0.000000,276.923077,123.076923
MA,14.285714,100.000000,85.714286
LT,2000.000000,0.000000,0.000000
VG,0.000000,1800.000000,0.000000
ER,0.000000,0.000000,1200.000000


Make pretty

In [8]:
df_direct = direct_allocations(df_direct)

| Departments   |            LT |            VG |            ER |         Total |
|:--------------|--------------:|--------------:|--------------:|--------------:|
| HR            |       640.000 |        80.000 |        80.000 |       800.000 |
| PD            |       525.000 |         0.000 |        75.000 |       600.000 |
| IT            |         0.000 |       276.923 |       123.077 |       400.000 |
| MA            |        14.286 |       100.000 |        85.714 |       200.000 |
| LT            | 2,000.000     |         0.000 |         0.000 | 2,000.000     |
| VG            |         0.000 | 1,800.000     |         0.000 | 1,800.000     |
| ER            |         0.000 |         0.000 | 1,200.000     | 1,200.000     |
| Total         | 3,179.286     | 2,256.923     | 1,563.791     | 7,000.000     |


### Step-down method  ###

---

**The weight matrix:**

Make a matrix with the weights from the lower rows of the table.

In [9]:
df_wght_matrix = dfs[dept_names].loc[dept_names]
diagonal = list(np.diag(df_wght_matrix))  # Keep the diagonal, we need it later

df_wght_matrix.style.background_gradient(axis=None, vmin=-1, vmax=1, cmap="Blues").format(precision=2)

,HR,PD,IT,MA,LT,VG,ER
Departments,,,,,,,
HR,-1.00,0.15,0.25,0.10,0.40,0.05,0.05
PD,0.25,-1.00,0.20,0.15,0.35,0.00,0.05
IT,0.20,0.10,-1.00,0.05,0.00,0.45,0.20
MA,0.10,0.05,0.15,-1.00,0.05,0.35,0.30
LT,0.00,0.00,0.00,0.00,1.00,0.00,0.00
VG,0.00,0.00,0.00,0.00,0.00,1.00,0.00
ER,0.00,0.00,0.00,0.00,0.00,0.00,1.00


We are not interested in the weights below the diagonal, because we use the **step-down** method. Let's also add a column of Totals, which enables re-weighing:

In [10]:
df_wght_matrix = df_wght_matrix.where(np.triu(np.ones(df_wght_matrix.shape)).astype(bool))
np.fill_diagonal(df_wght_matrix.values, np.NaN)
df_wght_matrix = df_wght_matrix.assign(Total=df_wght_matrix.sum(axis=1))
df_wght_matrix.style.background_gradient(axis=None, vmin=-1, vmax=1, cmap="Blues").format(precision=3)

,HR,PD,IT,MA,LT,VG,ER,Total
Departments,,,,,,,,
HR,nan,0.150,0.250,0.100,0.400,0.050,0.050,1.000
PD,nan,nan,0.200,0.150,0.350,0.000,0.050,0.750
IT,nan,nan,nan,0.050,0.000,0.450,0.200,0.700
MA,nan,nan,nan,nan,0.050,0.350,0.300,0.700
LT,nan,nan,nan,nan,nan,0.000,0.000,0.000
VG,nan,nan,nan,nan,nan,nan,0.000,0.000
ER,nan,nan,nan,nan,nan,nan,nan,0.000


We now re-weigh so that all rows add up to one (or zero depending on the diagonal values). We also restore the original diagonal values.

In [11]:
df_wght_matrix = df_wght_matrix[dept_names].div(df_wght_matrix["Total"], axis=0)
np.fill_diagonal(df_wght_matrix.values, diagonal)

df_wght_matrix.style.background_gradient(axis=None, vmin=-1, vmax=1, cmap="Blues").format(precision=3)

,HR,PD,IT,MA,LT,VG,ER
Departments,,,,,,,
HR,-1.000,0.150,0.250,0.100,0.400,0.050,0.050
PD,nan,-1.000,0.267,0.200,0.467,0.000,0.067
IT,nan,nan,-1.000,0.071,0.000,0.643,0.286
MA,nan,nan,nan,-1.000,0.071,0.500,0.429
LT,nan,nan,nan,nan,1.000,nan,nan
VG,nan,nan,nan,nan,nan,1.000,nan
ER,nan,nan,nan,nan,nan,nan,1.000


**The cost matrix**

Before we finish the weight matrix we need a cost matrix based on the weight matrix, to get the updated cost vector. The cost matrix only features the blue cells of the weight matrix. This will allow us to "trickle down" the costs of upstream departments.

In [12]:
df_cost_matrix = df_wght_matrix.copy()
np.fill_diagonal(df_cost_matrix.values, np.NaN)
df_cost_matrix.style.background_gradient(axis=None, vmin=-1, vmax=1, cmap="Blues").format(precision=2)

,HR,PD,IT,MA,LT,VG,ER
Departments,,,,,,,
HR,nan,0.15,0.25,0.10,0.40,0.05,0.05
PD,nan,nan,0.27,0.20,0.47,0.00,0.07
IT,nan,nan,nan,0.07,0.00,0.64,0.29
MA,nan,nan,nan,nan,0.07,0.50,0.43
LT,nan,nan,nan,nan,nan,nan,nan
VG,nan,nan,nan,nan,nan,nan,nan
ER,nan,nan,nan,nan,nan,nan,nan


We now have a matrix that allows us to update the original cost vector.

The cells in blue will be multiplied by the (updated) cost vector. So for IT we will multiply 0.25 and 0.26666667 by the relevant costs of the cost vector. We cannot use the original cost vetor values, because each department after the first will see its costs increased by the allocation of cost from upstream departments.

Separately, to finish the weight matrix for the final calculations, we should fill its blanks with zeroes:

In [13]:
df_wght_matrix = df_wght_matrix.fillna(0)
df_wght_matrix.style.background_gradient(axis=None, vmin=-1, vmax=1, cmap="Blues").format(precision=2)

,HR,PD,IT,MA,LT,VG,ER
Departments,,,,,,,
HR,-1.00,0.15,0.25,0.10,0.40,0.05,0.05
PD,0.00,-1.00,0.27,0.20,0.47,0.00,0.07
IT,0.00,0.00,-1.00,0.07,0.00,0.64,0.29
MA,0.00,0.00,0.00,-1.00,0.07,0.50,0.43
LT,0.00,0.00,0.00,0.00,1.00,0.00,0.00
VG,0.00,0.00,0.00,0.00,0.00,1.00,0.00
ER,0.00,0.00,0.00,0.00,0.00,0.00,1.00


Back to the cost vector. We should update it because the step-down method moves cost from one department to one lower down.

HR, for example moves 120 to PD (800 x 0.15), thus making that department allocate 720, not 600.

HR allocates 200 to IT (800 x 0.25) and PD allocates 192 to IT too (720 x 0.26666667). As a result, total cost for IT are 792, not 400.

The next function updates the cost vector for each of the upstream departments.

In [14]:
def new_cost(df, upstream, cost_vector):
    cost_vector = cost_vector.copy()
    for department in df[upstream]:
        init_cost = cost_vector.loc[department]  # original cost of department
        weight = df[department].dropna().values  # get the relevant weights from a department from the cost matrix
        if len(weight) > 0:
            cost = cost_vector.values  # get the entire cost vector
            cost = cost[0:upstream.index(department)]   # use only the costs that can be matched with the keys
            add_cost = cost @ weight  # calculate the additional cost for the department
        else:
            add_cost = 0
        cost_vector.loc[department] = init_cost + add_cost  # update the cost vector
    return cost_vector

df_upd_cost = new_cost(df_cost_matrix, upstream, df_cost)
df_cost.to_frame().join(df_upd_cost, rsuffix="_new")

,Cost,Cost_new
HR,800.0,800.000000
PD,600.0,720.000000
IT,400.0,792.000000
MA,200.0,480.571429
LT,2000.0,2000.000000
VG,1800.0,1800.000000
ER,1200.0,1200.000000


The final step here is to allocate the costs using the weight matrix and the updated cost vector.

In [15]:
df_step_down = df_wght_matrix.mul(df_upd_cost, axis=0)  # .where(np.triu(np.ones(df_wght_matrix.shape)).astype(bool))
df_step_down = total_allocations(df_step_down, upstream)
df_step_down.style.pipe(make_prettier)

,HR,PD,IT,MA,LT,VG,ER,Total of rows
Departments,,,,,,,,
HR,-800.00,120.00,200.00,80.00,320.00,40.00,40.00,0.00
PD,0.00,-720.00,192.00,144.00,336.00,0.00,48.00,-0.00
IT,0.00,0.00,-792.00,56.57,0.00,509.14,226.29,0.00
MA,0.00,0.00,0.00,-480.57,34.33,240.29,205.96,0.00
LT,0.00,0.00,0.00,0.00,2000.00,0.00,0.00,2000.00
VG,0.00,0.00,0.00,0.00,0.00,1800.00,0.00,1800.00
ER,0.00,0.00,0.00,0.00,0.00,0.00,1200.00,1200.00
Total,0.00,0.00,0.00,0.00,2690.33,2589.43,1720.24,7000.00


Compare to the previous allocation using the direct method.

In [16]:
df_step_down[downstream].join(df_direct[downstream], rsuffix="_stepped").style.pipe(make_prettier)  # df_direct.style.pipe(make_prettier)

,LT,VG,ER,LT_stepped,VG_stepped,ER_stepped
Departments,,,,,,
HR,320.00,40.00,40.00,640.00,80.00,80.00
PD,336.00,0.00,48.00,525.00,0.00,75.00
IT,0.00,509.14,226.29,0.00,276.92,123.08
MA,34.33,240.29,205.96,14.29,100.00,85.71
LT,2000.00,0.00,0.00,2000.00,0.00,0.00
VG,0.00,1800.00,0.00,0.00,1800.00,0.00
ER,0.00,0.00,1200.00,0.00,0.00,1200.00
Total,2690.33,2589.43,1720.24,3179.29,2256.92,1563.79


### Reciprocal method  ###

---

Like with the step-down method, we recalculate the cost vectors. This time using matrix algebra:

<br>

$X = C + KX$

where $X$ is the vector of true costs of the department, $C$ is the vector of upstream costs and $K$ is the upstream matrix of weights, transposed.

What the above formula says: the true cost of a department are the original costs of that department **plus** the costs that other departments allocate to that department. Remember: all departments allocate costs to all departments.

To give you an idea why this matrix notation: the true cost of HR ($X_{HR}$) are the original cost of HR ($C_{HR}$) plus share of the true costs that other upstream departments allocate to HR: $KX$.

We can solve this in one go by rewriting $X = C + KX$ into $IX = C + KX$, where $I$ is a matrix with ones on the diagonal and zeros otherwise.

Rearrange: $(I-K)X = C$

And solve: $X = (I-K)^{-1}C$.

We therefore only need the transposed upstream weight matrix ($K$), the original cost vector $C$, and some matrix magic to solve for the true costs of the upstream departments.

Note that there are two approaches here. One is where the weight matrix includes deliveries of services to self. The diagonal and the upper left corner then features the percentage of services delivered internally. The second approach is the one where all costs are downstreamed.  The upper left corner and the diagonal of the weight matrix then feature values of -1, that is, at least for the upstream departments.

Each of these approaches require a different matrix-equation to solve. The first approach solves for an adjusted negative transposed upstream weight matrix $(K \times -1 + I)$ and $C$, where $I$ is a matrix with only ones on the diagonal. The second approach solves for the negative transposed upstream weight matrix $(K \times -1)$ and $C$.

In [17]:
def get_matrix(df, lyst):
    if df._get_value(lyst[0], lyst[0]) >= 0:
        # Zimmerman's approach (first)
        M = df[lyst].loc[lyst].T.mul(-1.0).add(np.diag([1] * len(lyst)))
    else:
        # Approach with -1 in left upper corner (Second)
        M = df[lyst].loc[lyst].T.mul(-1.0)
    return M

M = get_matrix(dfs, upstream)
M

Departments,HR,PD,IT,MA
HR,1.00,-0.25,-0.20,-0.10
PD,-0.15,1.00,-0.10,-0.05
IT,-0.25,-0.20,1.00,-0.15
MA,-0.10,-0.15,-0.05,1.00


In [18]:
# Solve M and cost
df_reci = pd.Series(np.linalg.solve(M, df_cost[upstream]), index=upstream, name="Cost")
df_reci

HR    1275.935639
PD     914.985036
IT     979.049990
MA     513.793819
Name: Cost, dtype: float64

In [19]:
# compare original and true cost

df_reci_cost = pd.concat([df_reci, df_cost[downstream]])
pd.concat([df_cost, df_reci_cost], axis=1)

,Cost,Cost
HR,800.0,1275.935639
PD,600.0,914.985036
IT,400.0,979.049990
MA,200.0,513.793819
LT,2000.0,2000.000000
VG,1800.0,1800.000000
ER,1200.0,1200.000000


Get the full weight matrix:

In [20]:
df_wght_matrix = dfs[dept_names].loc[dept_names]
df_wght_matrix

,HR,PD,IT,MA,LT,VG,ER
Departments,,,,,,,
HR,-1.00,0.15,0.25,0.10,0.40,0.05,0.05
PD,0.25,-1.00,0.20,0.15,0.35,0.00,0.05
IT,0.20,0.10,-1.00,0.05,0.00,0.45,0.20
MA,0.10,0.05,0.15,-1.00,0.05,0.35,0.30
LT,0.00,0.00,0.00,0.00,1.00,0.00,0.00
VG,0.00,0.00,0.00,0.00,0.00,1.00,0.00
ER,0.00,0.00,0.00,0.00,0.00,0.00,1.00


And multiply by the true cost vector

In [21]:
df_reciprocal = df_wght_matrix.mul(df_reci_cost, axis=0)
df_reciprocal = total_allocations(df_reciprocal, upstream)
df_reciprocal.style.pipe(make_prettier)

,HR,PD,IT,MA,LT,VG,ER,Total of rows
Departments,,,,,,,,
HR,-1275.94,191.39,318.98,127.59,510.37,63.80,63.80,0.00
PD,228.75,-914.99,183.00,137.25,320.24,0.00,45.75,-0.00
IT,195.81,97.90,-979.05,48.95,0.00,440.57,195.81,0.00
MA,51.38,25.69,77.07,-513.79,25.69,179.83,154.14,-0.00
LT,0.00,0.00,0.00,0.00,2000.00,0.00,0.00,2000.00
VG,0.00,0.00,0.00,0.00,0.00,1800.00,0.00,1800.00
ER,0.00,0.00,0.00,0.00,0.00,0.00,1200.00,1200.00
Total,0.00,0.00,0.00,0.00,2856.31,2484.20,1659.49,7000.00


In [22]:
df_step_down.style.pipe(make_prettier)

,HR,PD,IT,MA,LT,VG,ER,Total of rows
Departments,,,,,,,,
HR,-800.00,120.00,200.00,80.00,320.00,40.00,40.00,0.00
PD,0.00,-720.00,192.00,144.00,336.00,0.00,48.00,-0.00
IT,0.00,0.00,-792.00,56.57,0.00,509.14,226.29,0.00
MA,0.00,0.00,0.00,-480.57,34.33,240.29,205.96,0.00
LT,0.00,0.00,0.00,0.00,2000.00,0.00,0.00,2000.00
VG,0.00,0.00,0.00,0.00,0.00,1800.00,0.00,1800.00
ER,0.00,0.00,0.00,0.00,0.00,0.00,1200.00,1200.00
Total,0.00,0.00,0.00,0.00,2690.33,2589.43,1720.24,7000.00


In [23]:
df_all = pd.concat([df_direct.loc['Total'], df_step_down.loc['Total'], df_reciprocal.loc['Total']], axis=1).dropna().loc[downstream]
df_all.columns=['Direct', 'Step-down','Reciprocal']
df_all.style.background_gradient(axis=None, vmin=0, vmax=5, cmap="Greys").format(precision=6)

,Direct,Step-down,Reciprocal
LT,3179.285714,2690.326531,2856.308709
VG,2256.923077,2589.428571,2484.197114
ER,1563.791209,1720.244898,1659.494177
